In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import pardir, path
import re

In [2]:
si_df = pd.read_json(path.join(pardir, 'full_si_data.json'))

In [3]:
temp = si_df.description[0]
temp

['<p>Life taxonomy ของเรา อ้างอิงตาม "Catalogue of Life" ( <a href="http://www.catalogueoflife.org/annual-checklist/2010/browse/tree">http://www.catalogueoflife.org/annual-checklist/2010/browse/tree</a>) โดยมี 7 อาณาจักร คือ Archaea, Bacteria, Animalia, Chromista, Fungi, Plantae และ Protozoa ใน 3 โดเมน (domain)<br/><br/>วิธีการใช้ ดัชนีสิ่งมีชีวิต มีสองรูปแบบ<br/>1. คือกดที่ Folder ทางด้านซ้ายมือไปเรื่อยๆ จนถึงสิ่งมีชีวิตที่คุณต้องการ หรือ<br/>2. ใส่ชื่อ หรือ คำใดๆที่คุณต้องการค้นหา จะเป็น ชื่อไทย ชื่อสามัญ หรือ ชื่อวิทยาศาสตร์ หรือ ข้อความเช่น เฉพาะถิ่นของไทย สัตว์คุ้มครอง ป่าพรุ หรือ ภาคใต้ สิ่งมีชีวิตที่มีคำเหล่านี้อยู่ในเนื้อความก็จะปรากฏขึ้นในกล่องทางด้านขวา (ที่ท่านกำลังอ่านอยู่นี้)\xa0<br/><br/>แต่ถ้าหากคุณมีภาพสิ่งมีชีวิต แต่ไม่แน่ใจหรือไม่ทราบว่าชื่ออะไร นำภาพไปแปะถามที่ บอร์ดพูดคุย จะง่ายกว่าครับ</p>',
 '<p class="child-help">*<u>หมายเหตุ</u> เมื่อกดที่รูปภาพ จะไปยังหน้าดัชนีนั้นๆทันที</p>']

In [9]:
def removeDescriptionArtifacts(rawDescriptionList):
    def basefunc(rawDescription):
        return rawDescription
        # remove any html tag
        # remove breakline
        # remove space after opening bracket
        # remove space before closing bracket    
        reg = re.sub('&amp;', '&',
                re.sub('[ ]+\)', ')', 
                re.sub('\([ ]+', '(', 
                re.sub('</?[a-z]+( href=".*")?( class=".*")?( style=".*")?( alt=".*")?( src=".*")?/?>', '', 
                re.sub('<br/>+', '\n', rawDescription)
              ))))
        return reg
    
    return '\n'.join(list(map(basefunc, rawDescriptionList)))

In [10]:
for i in si_df.description[0:100]:
    if i:
        print(removeDescriptionArtifacts(i))
        print('----')

<p>Life taxonomy ของเรา อ้างอิงตาม "Catalogue of Life" ( <a href="http://www.catalogueoflife.org/annual-checklist/2010/browse/tree">http://www.catalogueoflife.org/annual-checklist/2010/browse/tree</a>) โดยมี 7 อาณาจักร คือ Archaea, Bacteria, Animalia, Chromista, Fungi, Plantae และ Protozoa ใน 3 โดเมน (domain)<br/><br/>วิธีการใช้ ดัชนีสิ่งมีชีวิต มีสองรูปแบบ<br/>1. คือกดที่ Folder ทางด้านซ้ายมือไปเรื่อยๆ จนถึงสิ่งมีชีวิตที่คุณต้องการ หรือ<br/>2. ใส่ชื่อ หรือ คำใดๆที่คุณต้องการค้นหา จะเป็น ชื่อไทย ชื่อสามัญ หรือ ชื่อวิทยาศาสตร์ หรือ ข้อความเช่น เฉพาะถิ่นของไทย สัตว์คุ้มครอง ป่าพรุ หรือ ภาคใต้ สิ่งมีชีวิตที่มีคำเหล่านี้อยู่ในเนื้อความก็จะปรากฏขึ้นในกล่องทางด้านขวา (ที่ท่านกำลังอ่านอยู่นี้) <br/><br/>แต่ถ้าหากคุณมีภาพสิ่งมีชีวิต แต่ไม่แน่ใจหรือไม่ทราบว่าชื่ออะไร นำภาพไปแปะถามที่ บอร์ดพูดคุย จะง่ายกว่าครับ</p>
<p class="child-help">*<u>หมายเหตุ</u> เมื่อกดที่รูปภาพ จะไปยังหน้าดัชนีนั้นๆทันที</p>
----
<p>โดนเมนอาร์เคีย (archaebacteria)</p>
----
<p class="child-help">*<u>หมายเหตุ</u> เมื่อกดท